## Code for LLaMa-LORA Tuning

Find the code for tuning the LLaMa-7B mode using the LORA method below. It is an implementation of LLaMA-tuninig which is "fully open source under the Apache 2.0 license."

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
!git clone https://github.com/Lightning-AI/lit-llama ## this first ?!

Cloning into 'lit-llama'...
remote: Enumerating objects: 1710, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1710 (delta 46), reused 54 (delta 25), pack-reused 1615
Receiving objects: 100% (1710/1710), 670.91 KiB | 6.64 MiB/s, done.
Resolving deltas: 100% (1052/1052), done.


In [ ]:
!git clone https://huggingface.co/openlm-research/open_llama_7b_700bt_preview checkpoints/open-llama/7B

Cloning into 'checkpoints/open-llama/7B'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 498.91 KiB | 3.39 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 21.80 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
!pip install -r lit-llama/requirements.txt
# might take some time dpeneding on internet connectivity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Lightning-AI/lightning (to revision master) to /tmp/pip-install-l2p8xyrg/lightning_b593f0223a904f6c9d1c59767d4b4312
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-l2p8xyrg/lightning_b593f0223a904f6c9d1c59767d4b4312
  Resolved https://github.com/Lightning-AI/lightning to commit ca30fd7752582201a3966806c92e3acbbaf2a045
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .not

In [ ]:
!python lit-llama/scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/7B --model_size 7B

Initializing lit-llama
Saving to disk at checkpoints/lit-llama/7B
Processing checkpoints/open-llama/7B/pytorch_model-00002-of-00002.bin
Processing checkpoints/open-llama/7B/pytorch_model-00001-of-00002.bin


## Finetuning try 1

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
# connecting to drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


### Loading in data from drive and setting up own dataloader (.pt) files like the function in the python file prepare_alpacy do.

In [ ]:

import pandas as pd
airbnb_london_filtered_images = pd.read_csv("/content/gdrive/My Drive/Thesis/London_Data/discriminator_tabular_data_images.csv")

In [ ]:
import numpy as np
frac_train_size = 0.7

train_eval_airbnb_london_filtered_images = airbnb_london_filtered_images[airbnb_london_filtered_images.in_top_third == 1]

train_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.sample(n = int(np.ceil(frac_train_size*train_eval_airbnb_london_filtered_images.shape[0])), random_state = 100)
eval_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.drop(train_airbnb_london_filtered_advanced.index, axis = 0)

train_airbnb_london_filtered_advanced.index = list(range(train_airbnb_london_filtered_advanced.shape[0]))
eval_airbnb_london_filtered_advanced.index = list(range(eval_airbnb_london_filtered_advanced.shape[0]))


Logical dependencies of functions in prepare_alpaca file

'prepare' uses
> 'prepare_sample' uses
>> 'tokenzize' & 'generate promt'



In [ ]:
## Using the following instruction:

instruction = "Summarize the following description into a short title for an AirBnB listing."


In [ ]:
import os
from pathlib import Path
from typing import Optional
import torch
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer
import sys
import torch
import requests
import json
from torch.utils.data import random_split
from tqdm import tqdm

class Tokenizer:
    """Tokenizer for LLaMA."""

    def __init__(self, model_path: Path) -> None:
        self.processor = SentencePieceProcessor(model_file=str(model_path))
        self.bos_id = self.processor.bos_id()
        self.eos_id = self.processor.eos_id()
        self.pad_id = self.processor.pad_id()

    @property
    def vocab_size(self) -> int:
        return self.processor.vocab_size()

    def encode(
        self,
        string: str,
        bos: bool = True,
        eos: bool = False,
        max_length: int = -1,
        pad: bool = False,
        device: Optional[torch.device] = None
    ) -> torch.Tensor:
        tokens = self.processor.encode(string)
        if bos:
            tokens = [self.bos_id] + tokens
        if eos:
            tokens = tokens + [self.eos_id]
        if max_length > 0:
            tokens = tokens[:max_length]
        if pad and len(tokens) < max_length:
            tokens += [self.pad_id] * (max_length - len(tokens))

        return torch.tensor(tokens, dtype=torch.int, device=device)

    def decode(self, tokens: torch.Tensor) -> str:
        return self.processor.decode(tokens.tolist())

    @staticmethod
    def train(input: str, destination: str, vocab_size=32000) -> None:
        model_prefix = os.path.join(destination, "tokenizer")
        SentencePieceTrainer.Train(input=input, model_prefix=model_prefix, vocab_size=vocab_size)


## general function
def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)

## general function

def generate_prompt(example):
    """Generates a standardized message to prompt the model with an instruction, optional input and a
    'response' field."""

    if example["input"]:
        return (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"
        )
    return (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{example['instruction']}\n\n### Response:"
    )


def prepare_sample(example: dict, tokenizer: Tokenizer, max_length: int, mask_inputs: bool = True):
    """Processes a single sample.

    Each sample in the dataset consists of:
    - instruction: A string describing the task
    - input: A string holding a special input value for the instruction.
        This only applies to some samples, and in others this is empty.
    - output: The response string

    This function processes this data to produce a prompt text and a label for
    supervised training. The input text is formed as a single message including all
    the instruction, the input (optional) and the response.
    The label/target is the same message but can optionally have the instruction + input text
    masked out (mask_inputs=True).

    Finally, both the prompt and the label get tokenized. If desired, all tokens
    in the label that correspond to the original input prompt get masked out (default).
    """

    full_prompt = generate_prompt(example)
    full_prompt_and_response = full_prompt + example["output"]
    encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=max_length, eos=False)
    encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=max_length)

    # The labels are the full prompt with response, but with the prompt masked out
    labels = encoded_full_prompt_and_response.clone()
    if mask_inputs:
        labels[:len(encoded_full_prompt)]  = -1 ## corresponds to "ignore index" inprepare_alpaca.py file


    return {**example, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels}




#### prepare function is not required!

In [ ]:
tokenizer = Tokenizer("/content/checkpoints/lit-llama/tokenizer.model")

Reocding the prepare function:

In [ ]:
## setting up new dict

data_dict_train  = []
data_dict_eval  = []

In [ ]:

for i in range(train_airbnb_london_filtered_advanced.shape[0]):

  data_dict_train.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   train_airbnb_london_filtered_advanced.description[i],
      "output": train_airbnb_london_filtered_advanced.name[i]
  })

for i in range(eval_airbnb_london_filtered_advanced.shape[0]):

  data_dict_eval.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   eval_airbnb_london_filtered_advanced.description[i],
      "output": eval_airbnb_london_filtered_advanced.name[i]
  })


Create a new folder called "data"

In [ ]:
!mkdir data

In [ ]:
print("Processing train split ...")
max_seq_length = 256
mask_inputs = True
train_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_train]
torch.save(train_set, "/content/data/train.pt")




Processing train split ...


In [ ]:
print("Processing test split ...")
max_seq_length = 256
mask_inputs = True
test_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_eval]
torch.save(train_set, "/content/data/test.pt")


Processing test split ...


In [ ]:
epoch_size = len(train_set)
epoch_size


4048

#### Actual fine-tuning

The finetuning file is called. However, a few changes need to be made to this file (lora.py)

* in line 39, the size must be set to 4048 (size of my training set)
* in line 42, the lora_r hyper-param can be adjusted. I kept it at r = 8 as was suggested by LORA authors!
* in line 49, 'data_dict' must be set to "data"
* in line 52, 'out_dir' can be set to "out/adapter/airbnb" (Not necessary, just nicer!)

In [ ]:
!python lit-llama/finetune/lora.py

Global seed set to 1337
iter 0: loss 4.8885, time: 1971.93ms
iter 1: loss 3.6379, time: 243.85ms
iter 2: loss 3.2024, time: 267.48ms
iter 3: loss nan, time: 245.99ms
iter 4: loss 3.7454, time: 241.74ms
iter 5: loss 2.9494, time: 245.66ms
iter 6: loss 3.5960, time: 243.25ms
iter 7: loss 4.7820, time: 242.33ms
iter 8: loss 4.1417, time: 241.68ms
iter 9: loss 4.1249, time: 244.49ms
iter 10: loss 5.3503, time: 242.76ms
iter 11: loss 4.1460, time: 242.69ms
iter 12: loss 3.1389, time: 243.62ms
iter 13: loss 3.4740, time: 242.35ms
iter 14: loss 4.7652, time: 243.10ms
iter 15: loss 4.4754, time: 241.69ms
iter 16: loss 3.3450, time: 242.08ms
iter 17: loss 3.5197, time: 241.66ms
iter 18: loss 6.0065, time: 241.32ms
iter 19: loss 3.8131, time: 242.69ms
iter 20: loss 2.7893, time: 243.69ms
iter 21: loss 3.6749, time: 240.87ms
iter 22: loss 2.6452, time: 241.16ms
iter 23: loss 5.0563, time: 241.72ms
iter 24: loss 4.1168, time: 242.25ms
iter 25: loss 5.2320, time: 241.57ms
iter 26: loss 3.3037, time

## Evaluating and generating titles after the Fine-Tuning

Firstly: computation of ROUGE Scores

In [ ]:
import nltk
nltk.download('punkt')

def postprocess_text(text_sequence):   ##, labels):

    """
    Post-processing to prepare inputs to the ROGUE functions
    """

    text_sequence = [a.strip() for a in text_sequence]

    # ROUGE expects a newline after each sentence
    text_sequence = ["\n".join(nltk.sent_tokenize(a)) for a in text_sequence]

    return text_sequence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!mkdir out

In [ ]:
#import torch

model_ft = torch.load("/content/gdrive/My Drive/Thesis/Models/lit-llama-lora-finetuned.pth")
torch.save(model_ft, "/content/out/lit-llama-adapter-finetuned.pth")


In [ ]:
sys.path.append("/content/lit-llama")


import lightning as L
import torch
from generate import generate
from lit_llama import Tokenizer
from lit_llama.adapter import LLaMA
from lit_llama.utils import EmptyInitOnDevice, lazy_load, llama_model_lookup
from scripts.prepare_alpaca import generate_prompt

In [ ]:
adapter_path ="/content/out/lit-llama-adapter-finetuned.pth"
pretrained_path = "/content/checkpoints/lit-llama/7B/lit-llama.pth"
tokenizer_path = "/content/checkpoints/lit-llama/tokenizer.model"


fabric = L.Fabric(devices=1)
dtype = torch.bfloat16 if fabric.device.type == "cuda" and torch.cuda.is_bf16_supported() else torch.float32


The following function runs slowly, but sadly is needed to call for every generation ....

In [ ]:
def gen_title(pretrained_path, adapter_path, sample, max_new_tokens, top_k, temperature):

  """ Expects sample as set up above / like in the .pt data """

  with lazy_load(pretrained_path) as pretrained_checkpoint, lazy_load(adapter_path) as adapter_checkpoint:
      name = llama_model_lookup(pretrained_checkpoint)

      with EmptyInitOnDevice(
              device=fabric.device, dtype=dtype, quantization_mode = "llm.int8"
      ):
      #   quantization
          model = LLaMA.from_name(name)


      # 1. Load the pretrained weights
      model.load_state_dict(pretrained_checkpoint, strict=False)
      # 2. Load the fine-tuned adapter weights
      model.load_state_dict(adapter_checkpoint, strict=False)


  model.eval()
  model = fabric.setup_module(model)

  tokenizer = Tokenizer(tokenizer_path)

  prompt = generate_prompt(sample)
  encoded = tokenizer.encode(prompt, bos=True, eos=False, device=model.device)
  prompt_length = encoded.size(0)

  y = generate(model, encoded, max_new_tokens, temperature=temperature, top_k=top_k, eos_id=tokenizer.eos_id)

  output = tokenizer.decode(y)
  output = output.split("### Response:")[1].strip()

  return output


In [ ]:
## rouge with  jusst 1000 samples

test_set_ = test_set[:1000]


In [ ]:
## looping through eval/test samples

gen_titles_eval = []

for i in range(len(test_set_)):
  sample_ = test_set_[i]
  print(i/len(test_set_))
  gen_title_new = gen_title(pretrained_path, adapter_path, sample_, 30, 200, 0.2)
  gen_titles_eval.append(gen_title_new)



0.0


INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


0.006666666666666667


INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


0.013333333333333334


...

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


0.9866666666666667


INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


0.9933333333333333


INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


In [ ]:
## examining the generated titles in evaluation set!


for i in range(10):
  print(f"True title: {test_set[i]['output']}")
  print(f"Generated title: {gen_titles_eval[i]}")
  print("---------------------------------")


True title: Kew Gardens 3BR house in cul-de-sac
Generated title: 3 Bed House with garden, 20 min from Central London
---------------------------------
True title: Room with a view  zone 1  Central Bankside
Generated title: Separate bedroom with shared bathroom
---------------------------------
True title: Quiet Comfortable Room in Fulham
Generated title: Double Room in a Quiet House by the River
---------------------------------
True title: Beautiful, Luxurious Art Deco  +private bathroom
Generated title: Quiet, comfortable room in Hackney
---------------------------------
True title: Cosy Double studio in Zone 2 Hammersmith (3)
Generated title: Stunning studio flat in Hammersmith
---------------------------------
True title: ✦Bright Top Floor Sunset View Modern Chelsea apt✦
Generated title: Sunset view 1bed in Chelsea
---------------------------------
True title: West London-W7, Hanwell (Area Ealing) Room
Generated title: Double Room in Central West London
----------------------------

In [ ]:
titles_eval_true = [a["output"] for a in test_set_]

In [ ]:
processed_preds = postprocess_text(gen_titles_eval)
processed_labels = postprocess_text(titles_eval_true)


In [ ]:
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=04abdd9c0f77f508cf2e6f19d16a4c88d1e60aa36f1f035eaed198708d528555
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
#processed_preds_df = pd.read_csv("/content/gdrive/My Drive/Thesis/loss_data/preds_rouge.csv")
#processed_preds = processed_preds_df.preds



In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")

rouge_score.add_batch(predictions=processed_preds, references=processed_labels)

result = rouge_score.compute()

In [ ]:
print("The ROUGE metrics on the evaluation data AFTER LORA-PEFT are:")
print(f'     F1 - ROUGE-1: {np.round(result["rouge1"],4)}.')
print(f'     F1 - ROUGE-2: {np.round(result["rouge2"],4)}.')
print(f'     F1 - ROUGE-L: {np.round(result["rougeL"],4)}.')

The ROUGE metrics on the evaluation data AFTER LORA-PEFT are:
     F1 - ROUGE-1: 0.2524.
     F1 - ROUGE-2: 0.0882.
     F1 - ROUGE-L: 0.2423.


### Saving generated titles

Because this process is extremely slow (even with large A 100 GPU) I shall only generate titles for a subsample!

In [ ]:

sub_df = airbnb_london_filtered_images.sample(n = 1000)[["id", "description", "name"]]


In [ ]:
sub_df["gen_title"] = ""

In [ ]:
sub_df.index = list(range(sub_df.shape[0]))

In [ ]:
## setting up new dict

data_dict_sub_df = []

for i in range(sub_df.shape[0]):

  data_dict_sub_df.append({
      "instruction": "Summarize the following description into a short, appealing title for an AirBnB listing.",
      "input":   sub_df.description[i],
      "output": sub_df.name[i]
  })


max_seq_length = 256
mask_inputs = True
gen_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_sub_df]


In [ ]:
list_ids_ = list(sub_df.id)

In [ ]:
gen_titles_llama


gen_titles = []
list_ids = []

for i in range(len(gen_set)):
  sample_ = gen_set[i]
  print(i/len(gen_set))
  gen_title_new = gen_title(pretrained_path, adapter_path, sample_, 30, 200, 0.2)
  gen_titles.append(gen_title_new)
  list_ids.append(list_ids_[i])

  if i % 100 == 0:
    sub_df_lora = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
    sub_df_lora.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama.csv", index = False)

sub_df_lora = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
sub_df_lora.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama.csv", index = False)


In [ ]:
 # since this code in running overnight in Colab and no resources shall be wasted:

from google.colab import runtime
runtime.unassign()